In [1]:
import os
import sys
import random

import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

In [16]:
class DataGen(keras.utils.Sequence):
    def __init__(self, ids, path, batch_size=8, image_size=128):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()
        
    def __load__(self, id_name):
        image_path = os.path.join(self.path, id_name + "_out") + ".png"
        mask_path = os.path.join(self.path, "pr_masks", id_name + "_gth_hole_mask_0") + ".png"
        
        image = cv2.imread(image_path, 1)
        image = cv2.resize(image, (self.image_size, self.image_size))
        mask = np.zeros((self.image_size, self.image_size, 1))
        
        mask_im = cv2.imread(mask_path, 0)
        mask_im = cv2.bitwise_not(mask_im)
        mask_im = cv2.resize(mask_im, (self.image_size, self.image_size))
        
        mask = np.expand_dims(mask_im , axis=-1)
        
        image = image / 255.0
        mask = mask / 255.0
        
        return image, mask
    
    def __getitem__(self, index):
        if(index + 1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size
            
        files_batch = self.ids[index*self.batch_size : (index + 1)*self.batch_size]
        
        image = []
        mask = []
        
        for id_name in files_batch:
            _img, _mask = self.__load__(id_name)
            image.append(_img)
            mask.append(_mask)
            
            
        image = np.array(image)
        mask = np.array(mask)    
            
        return image, mask
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))

In [17]:
image_size = 128
train_path = "dataset/context-encoder/pack"
epochs = 10
batch_size = 5
val_data_size = 350

flist_path = train_path + "/flist2"
train_ids = open(flist_path).read().splitlines()
valid_ids = train_ids[:val_data_size]
all_ids = train_ids
train_ids = train_ids[val_data_size:]

In [18]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [23]:
def UNet():
    f = [16, 32, 64, 128, 256]
    inputs = keras.layers.Input((image_size, image_size, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

In [24]:
model = UNet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_76 (Conv2D)              (None, 128, 128, 16) 448         input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_77 (Conv2D)              (None, 128, 128, 16) 2320        conv2d_76[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_16 (MaxPooling2D) (None, 64, 64, 16)   0           conv2d_77[0][0]                  
____________________________________________________________________________________________

In [21]:
train_gen = DataGen(train_ids, train_path, image_size=image_size, batch_size=batch_size)
valid_gen = DataGen(valid_ids, train_path, image_size=image_size, batch_size=batch_size)

train_steps = len(train_ids)//batch_size
valid_steps = len(valid_ids)//batch_size

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=train_steps, validation_steps=valid_steps, epochs=epochs)

Epoch 1/10
130/130 [==============================] - 65s 496ms/step - loss: 0.1650 - acc: 0.8894 - val_loss: 0.0500 - val_acc: 0.9982
Epoch 2/10
130/130 [==============================] - 31s 239ms/step - loss: 0.0026 - acc: 0.9997 - val_loss: 2.9508e-05 - val_acc: 1.0000
Epoch 3/10
130/130 [==============================] - 31s 237ms/step - loss: 6.8866e-06 - acc: 1.0000 - val_loss: 5.8779e-07 - val_acc: 1.0000
Epoch 4/10
130/130 [==============================] - 31s 241ms/step - loss: 3.4333e-07 - acc: 1.0000 - val_loss: 3.1361e-07 - val_acc: 1.0000
Epoch 5/10
130/130 [==============================] - 31s 236ms/step - loss: 1.9009e-07 - acc: 1.0000 - val_loss: 2.0171e-07 - val_acc: 1.0000
Epoch 6/10
130/130 [==============================] - 30s 233ms/step - loss: 1.1564e-07 - acc: 1.0000 - val_loss: 1.4017e-07 - val_acc: 1.0000
Epoch 7/10
130/130 [==============================] - 32s 245ms/step - loss: 8.0387e-08 - acc: 1.0000 - val_loss: 1.0393e-07 - val_acc: 1.0000
Epoch 8/10


In [22]:
model.save_weights("Context_Encoder_UnetW_5_10.h5")